In [1]:
import pandas as pd
from multielo import MultiElo, Player, Tracker
import altair as alt

In [2]:
df_competition = pd.read_excel("C:/Users/ToniFORTORA/Downloads/DL_competitions.xlsx")
df_competitors = pd.read_excel("C:/Users/ToniFORTORA/Downloads/Top_list.xlsx")

In [3]:
#Create a flag where the date need to be cleaned
df_competition['two_dates'] = df_competition.Date.str[2] == '–'

#Sub dataframe with a copy of the date to be cleaned
df_duplicate = df_competition[df_competition['two_dates'] == True]

#Cleaning date in the 1st dataframe by keeping the first date
df_competition.loc[df_competition.two_dates==True, 'Date'] = df_competition.loc[df_competition.two_dates==True, 'Date'].str[3:]

#Cleaning date in the 2nd dataframe by keeping the second date
df_duplicate.Date = df_duplicate.Date.str[0:2] + ' ' + df_duplicate.Date.str[6:]

#Agrregating results
df_competition_clean = df_competition.append([df_duplicate],ignore_index=True)

#Merging competion and results based on the date and the place of the competion
df = pd.merge(df_competition_clean, df_competitors, how='inner', left_on=['Date', 'Venue'], right_on=['Date','Venue'])

#Pivoting table to have Position in the competition as columns and some cleaning
df_pivoted = df.pivot_table(index=['Date', 'Name', 'Country'], columns='Pos', values='Competitor', aggfunc=lambda x: ' '.join(x))
df_pivoted.reset_index(level=['Date', 'Name', 'Country'], inplace=True)
df_pivoted.rename(columns = {'Date':'date'}, inplace = True)
df_pivoted = df_pivoted[df_pivoted[2].notna()] #to be sure there is at least two finishers in the competion else the elo score can not be computed
df_pivoted.date = pd.to_datetime(df_pivoted.date)

#preparing dataframe tracker to use multielo package (keeping columns of interest and replacing NA by None)
#Careful here I only pick the first 10th finishers of a race 
df_tracker = df_pivoted[['date',1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
df_tracker = df_tracker.where(pd.notnull(df_pivoted), None)

#USing Multielo package to get rankings (maybe the score function need to be optimised -> see documentation)
tracker = Tracker(elo_rater=MultiElo(score_function_base=1.5))
tracker.process_data(df_tracker)
tracker.get_current_ratings()

C:\Users\ToniFORTORA\AppData\Local\Temp\ipykernel_5076\2222130741.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_duplicate.Date = df_duplicate.Date.str[0:2] + ' ' + df_duplicate.Date.str[6:]
C:\Users\ToniFORTORA\AppData\Local\Temp\ipykernel_5076\2222130741.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_competition_clean = df_competition.append([df_duplicate],ignore_index=True)


,rank,player_id,n_games,rating
0,1,Donavan BRAZIER,5,1198.309595
1,2,Nijel AMOS,6,1157.086799
2,3,Wyclife Kinyamal KISASY,6,1100.779620
3,4,Benjamin ROBERT,3,1083.475429
4,5,Ferguson Cheruiyot ROTICH,10,1057.772156
5,6,Slimane MOULA Ben PATTISON,1,1051.650113
6,7,Marco AROP,5,1045.191809
7,8,Bryce HOPPEL,1,1041.048337
8,9,Ferguson Cheruiyot ROTICH Jonathan KITILIT,1,1037.380268
9,10,Emmanuel WANYONYI,1,1029.215187


In [50]:
#Plotting the result

df_historical_rating = tracker.get_history_df()

highlight = alt.selection(type='single', on='mouseover', 
                          fields=['player_id'], nearest=True, empty="none")

base = alt.Chart(df_historical_rating
).encode(
    x=alt.X("date:T", axis=alt.Axis(format='%Y %B')),
    y=alt.Y("rating:Q", scale=alt.Scale(zero=False)),
    #color="player_id:N",
    tooltip = ["player_id","rating"]
)

points = base.mark_circle(point=True).encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=1350,
    height=500
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3)),
    color=alt.condition(highlight, 'player_id', alt.value("lightgray"), legend=None)
)

alt.layer(points, lines).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
)

alt.LayerChart(...)